## Minimal example of a Python Jupyter Notebook snippet that:

* Queries Wikidata for trail sections with their OSM relation IDs (wdt:P402)
* Fetches multilingual labels
* Uses requests to run the SPARQL query
* Visualizes results in a simple table
* Optionally, shows how you can link to the OSM relation on the map



In [1]:
import time

start_time = time.time()

In [2]:
# Make sure to install required package first if you don't have it
# !pip install pandas requests

import requests
import pandas as pd

# Define the SPARQL endpoint and query
endpoint_url = "https://query.wikidata.org/sparql"

query = """
SELECT ?section ?sectionLabel ?osmRelationID WHERE {
  ?section wdt:P361 wd:Q131318799 .   # Sections part of Stockholm Archipelago Trail
  ?section wdt:P402 ?osmRelationID .  # OSM relation ID property
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ?sectionLabel
"""

def run_sparql_query(endpoint_url, query):
    headers = {
        "Accept": "application/sparql-results+json"
    }
    r = requests.get(endpoint_url, params={'query': query}, headers=headers)
    r.raise_for_status()
    return r.json()

results = run_sparql_query(endpoint_url, query)

# Parse results into a DataFrame
data = []
for item in results['results']['bindings']:
    section = item['section']['value']
    label = item.get('sectionLabel', {}).get('value', '')
    osm_id = item['osmRelationID']['value']
    data.append({'Section': label, 'Wikidata URL': section, 'OSM Relation ID': osm_id})

df = pd.DataFrame(data)

# Show the table
df


,Section,Wikidata URL,OSM Relation ID
0,SAT Arholma,http://www.wikidata.org/entity/Q133374147,19012436
1,SAT Brottö,http://www.wikidata.org/entity/Q133724240,19141225
2,SAT Finnhamn,http://www.wikidata.org/entity/Q133724249,19018272
3,SAT Fjärdlång,http://www.wikidata.org/entity/Q133724237,19016280
4,SAT Furusund,http://www.wikidata.org/entity/Q133724254,19016187
5,SAT Grinda,http://www.wikidata.org/entity/Q133502246,19079703
6,SAT Ingmarsö,http://www.wikidata.org/entity/Q133724247,19080874
7,SAT Landsort,http://www.wikidata.org/entity/Q133449160,19013576
8,SAT Lidö,http://www.wikidata.org/entity/Q133374185,19020231
9,SAT Möja,http://www.wikidata.org/entity/Q133724252,19023630


In [3]:
df['OSM Link'] = df['OSM Relation ID'].apply(lambda x: f"https://www.openstreetmap.org/relation/{x}")
df.style.format({'OSM Link': lambda x: f'<a href="{x}" target="_blank">{x}</a>'})

,Section,Wikidata URL,OSM Relation ID,OSM Link
0,SAT Arholma,http://www.wikidata.org/entity/Q133374147,19012436,https://www.openstreetmap.org/relation/19012436
1,SAT Brottö,http://www.wikidata.org/entity/Q133724240,19141225,https://www.openstreetmap.org/relation/19141225
2,SAT Finnhamn,http://www.wikidata.org/entity/Q133724249,19018272,https://www.openstreetmap.org/relation/19018272
3,SAT Fjärdlång,http://www.wikidata.org/entity/Q133724237,19016280,https://www.openstreetmap.org/relation/19016280
4,SAT Furusund,http://www.wikidata.org/entity/Q133724254,19016187,https://www.openstreetmap.org/relation/19016187
5,SAT Grinda,http://www.wikidata.org/entity/Q133502246,19079703,https://www.openstreetmap.org/relation/19079703
6,SAT Ingmarsö,http://www.wikidata.org/entity/Q133724247,19080874,https://www.openstreetmap.org/relation/19080874
7,SAT Landsort,http://www.wikidata.org/entity/Q133449160,19013576,https://www.openstreetmap.org/relation/19013576
8,SAT Lidö,http://www.wikidata.org/entity/Q133374185,19020231,https://www.openstreetmap.org/relation/19020231
9,SAT Möja,http://www.wikidata.org/entity/Q133724252,19023630,https://www.openstreetmap.org/relation/19023630


### POIs with custom markers

In [4]:
import requests
import pandas as pd
import folium

# SPARQL endpoint
endpoint_url = "https://query.wikidata.org/sparql"

# Adapted SPARQL query for points
query = """
SELECT DISTINCT ?id ?idLabel ?geo ?hexcolor ?iconname WHERE {
  ?id wdt:P6104 wd:Q134294510 ;
       wdt:P625 ?geo .

  OPTIONAL {
    ?id wdt:P31 ?instance .
    ?instance wdt:P465 ?hexcolor .
    ?instance p:P1343 ?stmt .
    ?stmt pq:P1476 ?iconname .
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

# Fetch SPARQL data
headers = {"Accept": "application/sparql-results+json"}
r = requests.get(endpoint_url, params={'query': query}, headers=headers)
r.raise_for_status()
results = r.json()

# Parse to DataFrame
rows = []
for item in results['results']['bindings']:
    label = item.get('idLabel', {}).get('value', '')
    coords = item['geo']['value'].replace('Point(', '').replace(')', '')
    lon, lat = map(float, coords.split(' '))
    color = '#' + item.get('hexcolor', {}).get('value', '228b22')  # fallback green
    symbol = item.get('iconname', {}).get('value', 'landmark-JP')  # fallback symbol
    rows.append({'label': label, 'lat': lat, 'lon': lon, 'color': color, 'symbol': symbol})

df = pd.DataFrame(rows)

# Create map
m = folium.Map(location=[59.3, 18.5], zoom_start=9)

# Add points
for _, row in df.iterrows():
    folium.CircleMarker(
        location=(row['lat'], row['lon']),
        radius=6,
        color=row['color'],
        fill=True,
        fill_color=row['color'],
        tooltip=row['label']
    ).add_to(m)

m


### ipywidgets for language selection

In [5]:
import requests
import pandas as pd
import folium
from ipywidgets import interact, Dropdown

# Function to fetch and render the map
def render_map(language='en,sv'):
    # SPARQL query with dynamic language input
    query = f"""
    SELECT DISTINCT ?id ?idLabel ?geo ?hexcolor ?iconname WHERE {{
      ?id wdt:P6104 wd:Q134294510 ;
           wdt:P625 ?geo .

      OPTIONAL {{
        ?id wdt:P31 ?instance .
        ?instance wdt:P465 ?hexcolor .
        ?instance p:P1343 ?stmt .
        ?stmt pq:P1476 ?iconname .
      }}

      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "{language}" }}
    }}
    """

    # Fetch SPARQL data
    endpoint_url = "https://query.wikidata.org/sparql"
    headers = {"Accept": "application/sparql-results+json"}
    r = requests.get(endpoint_url, params={'query': query}, headers=headers)
    r.raise_for_status()
    results = r.json()

    # Parse to DataFrame
    rows = []
    for item in results['results']['bindings']:
        label = item.get('idLabel', {}).get('value', '')
        coords = item['geo']['value'].replace('Point(', '').replace(')', '')
        lon, lat = map(float, coords.split(' '))
        color = '#' + item.get('hexcolor', {}).get('value', '228b22')
        symbol = item.get('iconname', {}).get('value', 'landmark-JP')
        rows.append({'label': label, 'lat': lat, 'lon': lon, 'color': color, 'symbol': symbol})

    df = pd.DataFrame(rows)

    # Create map
    m = folium.Map(location=[59.3, 18.5], zoom_start=9)

    # Add points
    for _, row in df.iterrows():
        folium.CircleMarker(
            location=(row['lat'], row['lon']),
            radius=6,
            color=row['color'],
            fill=True,
            fill_color=row['color'],
            tooltip=row['label']
        ).add_to(m)

    return m

# Create interactive dropdown
lang_dropdown = Dropdown(
    options=[
        ('English', 'en'),
        ('Swedish', 'sv'),
        ('Finnish', 'fi'),
        ('Norwegian', 'no'),
        ('German', 'de'),
        ('French', 'fr'),
        ('Fallback (en,sv)', 'en,sv')
    ],
    value='en,sv',
    description='Language:',
)

# Display widget and map
interact(lambda language: render_map(language), language=lang_dropdown);


interactive(children=(Dropdown(description='Language:', index=6, options=(('English', 'en'), ('Swedish', 'sv')…

### Add description 

In [6]:
import requests
import pandas as pd
import folium
from ipywidgets import interact, Dropdown

# Function to fetch and render the map
def render_map(language='en,sv'):
    # SPARQL query with dynamic language input
    query = f"""
    SELECT DISTINCT ?id ?idLabel ?geo ?hexcolor ?iconname ?description ?svlink WHERE {{
      ?id wdt:P6104 wd:Q134294510 ;
           wdt:P625 ?geo .

      OPTIONAL {{
        ?id wdt:P31 ?instance .
        ?instance wdt:P465 ?hexcolor .
        ?instance p:P1343 ?stmt .
        ?stmt pq:P1476 ?iconname .
      }}

      OPTIONAL {{ ?id schema:description ?description FILTER(LANG(?description) = SUBSTR("{language}", 1, 2)) }}
      OPTIONAL {{ ?svlink schema:about ?id ; schema:isPartOf <https://sv.wikipedia.org/> }}

      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "{language}" }}
    }}
    """

    # Fetch SPARQL data
    endpoint_url = "https://query.wikidata.org/sparql"
    headers = {"Accept": "application/sparql-results+json"}
    r = requests.get(endpoint_url, params={'query': query}, headers=headers)
    r.raise_for_status()
    results = r.json()

    # Parse to DataFrame
    rows = []
    for item in results['results']['bindings']:
        label = item.get('idLabel', {}).get('value', '')
        coords = item['geo']['value'].replace('Point(', '').replace(')', '')
        lon, lat = map(float, coords.split(' '))
        color = '#' + item.get('hexcolor', {}).get('value', '228b22')
        symbol = item.get('iconname', {}).get('value', 'landmark-JP')
        description = item.get('description', {}).get('value', '')
        svlink = item.get('svlink', {}).get('value', '')

        tooltip_text = label
        if description:
            tooltip_text += f"\n{description}"
        if svlink:
            tooltip_text += f"\n[sv.wikipedia]({svlink})"

        rows.append({'label': tooltip_text, 'lat': lat, 'lon': lon, 'color': color, 'symbol': symbol})

    df = pd.DataFrame(rows)

    # Create map
    m = folium.Map(location=[59.3, 18.5], zoom_start=9)

    # Add points
    for _, row in df.iterrows():
        folium.CircleMarker(
            location=(row['lat'], row['lon']),
            radius=6,
            color=row['color'],
            fill=True,
            fill_color=row['color'],
            tooltip=row['label']
        ).add_to(m)

    return m

# Create interactive dropdown
lang_dropdown = Dropdown(
    options=[
        ('English', 'en'),
        ('Swedish', 'sv'),
        ('Finnish', 'fi'),
        ('Norwegian', 'no'),
        ('German', 'de'),
        ('French', 'fr'),
        ('Fallback (en,sv)', 'en,sv')
    ],
    value='en,sv',
    description='Language:',
)

# Display widget and map
interact(lambda language: render_map(language), language=lang_dropdown);


interactive(children=(Dropdown(description='Language:', index=6, options=(('English', 'en'), ('Swedish', 'sv')…

### Version 2 more languages   
* add the language name in the dropdown not just the code as "Swedish - sv"
* Filtering by instance types 
* Adding the actual trail geometry (via geoline)
* Add P18 image

In [13]:
import requests
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output

# SPARQL endpoint
endpoint_url = "https://query.wikidata.org/sparql"

# Language groups
swedish_official = {'sv', 'fi', 'se', 'me', 'ri', 'sm', 'fit', 'yi', 'rmy'}
nordic_languages = swedish_official.union({'da', 'no', 'nn', 'is', 'fo'})
european_languages = {
    'en', 'fr', 'de', 'es', 'it', 'pt', 'pl', 'nl', 'cs', 'hu', 'ro', 'bg', 'el', 'tr', 'et', 'lv', 'lt', 'sk', 'sl', 'hr', 'mt',
    'da', 'fi', 'sv', 'no', 'is'
}  

# Language display mapping
language_names = {
    'sv': 'Swedish', 'en': 'English', 'fr': 'French', 'de': 'German', 'nn': 'Norwegian Nynorsk', 'nb': 'Norwegian Bokmål',
    'it': 'Italian', 'pl': 'Polish', 'es': 'Spanish', 'pt': 'Portuguese', 'ar': 'Arabic', 'ru': 'Russian', 'zh': 'Chinese',
    'fi': 'Finnish', 'da': 'Danish', 'nl': 'Dutch', 'jp': 'Japanese', 'fa': 'Persian', 'uk': 'Ukrainian', 'ku': 'Kurdish',
    'fit': 'Meänkieli', 'se': 'Northern Sami', 'sma': 'Southern Sami', 'smj': 'Lule Sami', 'sje': 'Pite Sami', 'sju': 'Ume Sami',
    'rmy': 'Romani', 'yi': 'Yiddish', 'et': 'Estonian', 'lv': 'Latvian', 'lt': 'Lithuanian', 'cs': 'Czech', 'hu': 'Hungarian',
    'el': 'Greek', 'tr': 'Turkish', 'ko': 'Korean', 'hi': 'Hindi', 'th': 'Thai', 'vi': 'Vietnamese', 'he': 'Hebrew',
    'id': 'Indonesian', 'ms': 'Malay', 'is': 'Icelandic', 'fo': 'Faroese'
}

all_languages = list(language_names.keys())

# Autodetect browser/system language with fallback to Swedish
import locale

def detect_language():
    try:
        lang = locale.getlocale()[0]
    except:
        lang = None
    if lang and lang[:2] in all_languages:
        return lang[:2]
    return 'sv'

# Dropdown for language selection with full names
language_selector = widgets.Dropdown(
    options=sorted([(f"{language_names[code]} - {code}", code) for code in all_languages]),
    value=detect_language(),
    description='Language:',
    disabled=False
)


# Instance type list from user (simplified display, filtering using full list)
custom_instances = [
    ('All', ''),
    ('Systembolagets ombud', 'wd:Q134529187'),
    ('Apotek', 'wd:Q13107184'),
    ('Arbetslivsmuseum', 'wd:Q33506'),
    ('Badplats', 'wd:Q567998'),
    ('Bageri', 'wd:Q274393'),
    ('Bastu', 'wd:Q57036'),
    ('Begravningsplats', 'wd:Q39614'),
    ('Bensinstation', 'wd:Q205495'),
    ('Biograf', 'wd:Q41253'),
    ('Bondgård', 'wd:Q72030539'),
    ('Brygga', 'wd:Q133867301'),
    ('Butik', 'wd:Q213441'),
    ('By', 'wd:Q532'),
    ('Byggnad', 'wd:Q41176'),
    ('Campingplats', 'wd:Q27108230'),
    ('Cykeluthyrning', 'wd:Q134529179'),
    ('Dricksvatten', 'wd:Q7892'),
    ('Fyr', 'wd:Q39715'),
    ('Fågelstation', 'wd:Q1365207'),
    ('Glamping', 'wd:Q2153744'),
    ('Grav', 'wd:Q173387'),
    ('Grillplats', 'wd:Q1546788'),
    ('Grotta', 'wd:Q35509'),
    ('Gruva', 'wd:Q820477'),
    ('Gästhamn', 'wd:Q10512405'),
    ('Gård', 'wd:Q131596'),
    ('Gårdsbutik', 'wd:Q1371823'),
    ('Hamn', 'wd:Q44782'),
    ('Hamnkontor', 'wd:Q55076881'),
    ('Hembygdsgård', 'wd:Q10520688'),
    ('Hembygdsmuseum', 'wd:Q33506'),
    ('Hjärtstartare', 'wd:Q1450682'),
    ('Hotell', 'wd:Q27686'),
    ('Insjö', 'wd:Q23397'),
    ('Jordbruk', 'wd:Q11451'),
    ('Jungfrudans', 'wd:Q1937879'),
    ('Jättegryta', 'wd:Q1358604'),
    ('Kafé', 'wd:Q30022'),
    ('Kajakuthyrning', 'wd:Q134539211'),
    ('Kapell', 'wd:Q108325'),
    ('Kiosk', 'wd:Q693369'),
    ('Krog', 'wd:Q256020'),
    ('Kustartilleribatteri', 'wd:Q16536851'),
    ('Kyrka', 'wd:Q16970'),
    ('Kyrkogård', 'wd:Q39614'),
    ('Lanthandel', 'wd:Q1295201'),
    ('Livsmedelsbutik', 'wd:Q1295201'),
    ('Lotsstation', 'wd:Q16948701'),
    ('Minneslund', 'wd:Q39614'),
    ('Minnesmärke', 'wd:Q5003624'),
    ('Museijärnväg', 'wd:Q420962'),
    ('Museum', 'wd:Q33506'),
    ('Naturhamn', 'wd:Q283202'),
    ('Naturreservat', 'wd:Q179049'),
    ('Paviljong', 'wd:Q276173'),
    ('Pensionat', 'wd:Q1065252'),
    ('Pub', 'wd:Q212198'),
    ('Restaurang', 'wd:Q11707'),
    ('Rum och frukost', 'wd:Q27686'),
    ('Ryssugn', 'wd:Q10658341'),
    ('Samhälle', 'wd:Q486972'),
    ('Skola', 'wd:Q3914'),
    ('Skolenhet', 'wd:Q3914'),
    ('Småort i Sverige', 'wd:Q14839548'),
    ('Snorkelled', 'wd:Q134078772'),
    ('Staty', 'wd:Q179700'),
    ('Stugby', 'wd:Q1406318'),
    ('Stuguthyrning', 'wd:Q135107662'),
    ('Teaterkonst', 'wd:Q11635'),
    ('Telegraf', 'wd:Q6987428'),
    ('Turistattraktion', 'wd:Q570116'),
    ('Tältplats', 'wd:Q832778'),
    ('Tätort i Sverige', 'wd:Q12813115'),
    ('Udde', 'wd:Q191992'),
    ('Utegym', 'wd:Q692630'),
    ('Utsiktsplats', 'wd:Q6017969'),
    ('Vandrarhem', 'wd:Q654772'),
    ('Vandringsled', 'wd:Q2143825'),
    ('Vindskydd', 'wd:Q1797440'),
    ('Väderkvarn', 'wd:Q38720'),
    ('Vägvisare', 'wd:Q1937027')
]

instance_selector = widgets.Dropdown(
    options=custom_instances,
    value='',
    description='Instance:',
    disabled=False,
)

def fetch_and_show_map(lang='en', instance=''):
    filter_clause = f"?id wdt:P31 {instance} ." if instance else ""

    query = f"""
    SELECT DISTINCT ?id ?geo ?hexcolor ?iconname
           (SAMPLE(?labelLang) AS ?labelLangVal)
           (SAMPLE(?descLang) AS ?descLangVal)
           (SAMPLE(?svLabel) AS ?svLabelVal)
           (SAMPLE(?svdesc) AS ?svdescVal)
           (SAMPLE(?svwiki) AS ?svwikiVal)
           (SAMPLE(?img) AS ?imgVal)
           (SAMPLE(?p3749) AS ?p3749Val)
           (SAMPLE(?website) AS ?websiteVal)
           (SAMPLE(?facebook) AS ?facebookVal)
           (SAMPLE(?instagram) AS ?instagramVal)
           (SAMPLE(?location) AS ?locationVal)
    WHERE {{
      ?id wdt:P6104 wd:Q134294510 ;
           wdt:P625 ?geo .

      OPTIONAL {{
        ?id wdt:P31 ?instance .
        ?instance wdt:P465 ?hexcolor .
        ?instance p:P1343 ?stmt .
        ?stmt pq:P1476 ?iconname .
      }}

      OPTIONAL {{ ?id rdfs:label ?labelLang FILTER (lang(?labelLang) = '{lang}') }}
      OPTIONAL {{ ?id schema:description ?descLang FILTER (lang(?descLang) = '{lang}') }}
      OPTIONAL {{ ?id rdfs:label ?svLabel FILTER (lang(?svLabel) = 'sv') }}
      OPTIONAL {{ ?id schema:description ?svdesc FILTER (lang(?svdesc) = 'sv') }}
      OPTIONAL {{ ?svwiki schema:about ?id; schema:isPartOf <https://sv.wikipedia.org/> }}
      OPTIONAL {{ ?id wdt:P18 ?img }}
      OPTIONAL {{ ?id wdt:P3749 ?p3749 }}
      OPTIONAL {{ ?id wdt:P856 ?website }}
      OPTIONAL {{ ?id wdt:P2013 ?facebook }}
      OPTIONAL {{ ?id wdt:P2003 ?instagram }}
      OPTIONAL {{ ?id wdt:P4173 ?location }}
	  OPTIONAL {{
 		?id wdt:P2789 ?satSection .
  		?satSection rdfs:label ?satSectionLabel FILTER (lang(?satSectionLabel) = 'sv') .
    	OPTIONAL {{
    	    ?satSection p:P856 ?websiteStmt .
    	    ?websiteStmt ps:P856 ?satWebsite .
            OPTIONAL {{ ?websiteStmt pq:P407 ?websiteLang }}
        }}
      }}
      {filter_clause}
    }}
    GROUP BY ?id ?geo ?hexcolor ?iconname
    """

    headers = {"Accept": "application/sparql-results+json"}
    r = requests.get(endpoint_url, params={'query': query}, headers=headers)
    r.raise_for_status()
    results = r.json()

    rows = []
    for item in results['results']['bindings']:
        label = item.get('labelLangVal', {}).get('value', '-')
        description = item.get('descLangVal', {}).get('value', '')
        sv_label = item.get('svLabelVal', {}).get('value', '')
        sv_description = item.get('svdescVal', {}).get('value', '')
        coords = item['geo']['value'].replace('Point(', '').replace(')', '')
        lon, lat = map(float, coords.split(' '))
        color = '#' + item.get('hexcolor', {}).get('value', '228b22')
        symbol = item.get('iconname', {}).get('value', 'landmark-JP')
        img = item.get('imgVal', {}).get('value', '')
        wiki_url = item.get('svwikiVal', {}).get('value', '')
        qid = item.get('id', {}).get('value', '').split('/')[-1]
        gmaps = item.get('p3749Val', {}).get('value', '')
        website = item.get('websiteVal', {}).get('value', '')
        facebook = item.get('facebookVal', {}).get('value', '')
        instagram = item.get('instagramVal', {}).get('value', '')
        location = item.get('locationVal', {}).get('value', '')

        popup_content = f"<b>{lang}: {label} - {description}</b><br>"
        if lang != 'sv' and (sv_label or sv_description):
            popup_content += f"<b>sv: {sv_label} - {sv_description}</b><br>"
        popup_content += "<div style='margin-top:8px;'><b>Links:</b><br>"
        popup_content += f"<a href='https://www.wikidata.org/wiki/{qid}' title='Wikidata link' target='_blank'>🔗 Wikidata</a><br>"
        if gmaps:
            popup_content += f"<a href='https://maps.google.com/?cid={gmaps}' title='View on Google Maps' target='_blank'>🗺️ Google Maps</a><br>"
        if wiki_url:
            popup_content += f"<a href='{wiki_url}' target='_blank' title='Wikipedia (sv)'>📘 Wikipedia (sv)</a><br>"
        if website:
            popup_content += f"🌐 <a href='{website}' target='_blank' title='Website'>{website}</a><br>"
        if facebook:
            popup_content += f"📘 <a href='https://facebook.com/{facebook}' target='_blank' title='Facebook'>{facebook}</a><br>"
        if instagram:
            popup_content += f"📸 <a href='https://instagram.com/{instagram}' target='_blank' title='Instagram'>{instagram}</a>"
            if location:
                popup_content += f" — <a href='https://www.instagram.com/explore/locations/{location}' target='_blank' title='Instagram Location'>{location}</a>"
            popup_content += "<br>"
        if img:
            popup_content += f"<br><div style='margin-top:8px;'><b>Image:</b><br><img src='{img}' width='200' title='{label}' alt='{label}'></div>"

        rows.append({'label': popup_content, 'lat': lat, 'lon': lon, 'color': color, 'symbol': symbol})

    df = pd.DataFrame(rows)

    m = folium.Map(location=[59.3, 18.5], zoom_start=9)

    for _, row in df.iterrows():
        folium.CircleMarker(
            location=(row['lat'], row['lon']),
            radius=6,
            color=row['color'],
            fill=True,
            fill_color=row['color'],
            popup=row['label'],
            tooltip=row['label']
        ).add_to(m)

    display(m)


# Interactive update
widgets.interact(fetch_and_show_map, lang=language_selector, instance=instance_selector)


interactive(children=(Dropdown(description='Language:', index=37, options=(('Arabic - ar', 'ar'), ('Chinese - …

<function __main__.fetch_and_show_map(lang='en', instance='')>

### Add Trail


In [16]:
import requests
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output

# SPARQL endpoint
endpoint_url = "https://query.wikidata.org/sparql"

# Language groups
swedish_official = {'sv', 'fi', 'se', 'me', 'ri', 'sm', 'fit', 'yi', 'rmy'}
nordic_languages = swedish_official.union({'da', 'no', 'nn', 'is', 'fo'})
european_languages = {
    'en', 'fr', 'de', 'es', 'it', 'pt', 'pl', 'nl', 'cs', 'hu', 'ro', 'bg', 'el', 'tr', 'et', 'lv', 'lt', 'sk', 'sl', 'hr', 'mt',
    'da', 'fi', 'sv', 'no', 'is'
}  

# Language display mapping
language_names = {
    'sv': 'Swedish', 'en': 'English', 'fr': 'French', 'de': 'German', 'nn': 'Norwegian Nynorsk', 'nb': 'Norwegian Bokmål',
    'it': 'Italian', 'pl': 'Polish', 'es': 'Spanish', 'pt': 'Portuguese', 'ar': 'Arabic', 'ru': 'Russian', 'zh': 'Chinese',
    'fi': 'Finnish', 'da': 'Danish', 'nl': 'Dutch', 'jp': 'Japanese', 'fa': 'Persian', 'uk': 'Ukrainian', 'ku': 'Kurdish',
    'fit': 'Meänkieli', 'se': 'Northern Sami', 'sma': 'Southern Sami', 'smj': 'Lule Sami', 'sje': 'Pite Sami', 'sju': 'Ume Sami',
    'rmy': 'Romani', 'yi': 'Yiddish', 'et': 'Estonian', 'lv': 'Latvian', 'lt': 'Lithuanian', 'cs': 'Czech', 'hu': 'Hungarian',
    'el': 'Greek', 'tr': 'Turkish', 'ko': 'Korean', 'hi': 'Hindi', 'th': 'Thai', 'vi': 'Vietnamese', 'he': 'Hebrew',
    'id': 'Indonesian', 'ms': 'Malay', 'is': 'Icelandic', 'fo': 'Faroese'
}

all_languages = list(language_names.keys())

# Autodetect browser/system language with fallback to Swedish
import locale

def detect_language():
    try:
        lang = locale.getlocale()[0]
    except:
        lang = None
    if lang and lang[:2] in all_languages:
        return lang[:2]
    return 'sv'

# Dropdown for language selection with full names
language_selector = widgets.Dropdown(
    options=sorted([(f"{language_names[code]} - {code}", code) for code in all_languages]),
    value=detect_language(),
    description='Language:',
    disabled=False
)

# Instance selector (truncated for brevity, already populated above)
instance_selector = widgets.Dropdown(
    options=custom_instances,
    value='',
    description='Instance:',
    disabled=False,
)

def fetch_and_show_map(lang='en', instance=''):
    filter_clause = f"?id wdt:P31 {instance} ." if instance else ""

    query = f"""
    SELECT DISTINCT ?id ?geo ?hexcolor ?iconname
           (SAMPLE(?labelLang) AS ?labelLangVal)
           (SAMPLE(?descLang) AS ?descLangVal)
           (SAMPLE(?svLabel) AS ?svLabelVal)
           (SAMPLE(?svdesc) AS ?svdescVal)
           (SAMPLE(?svwiki) AS ?svwikiVal)
           (SAMPLE(?img) AS ?imgVal)
           (SAMPLE(?p3749) AS ?p3749Val)
           (SAMPLE(?website) AS ?websiteVal)
           (SAMPLE(?facebook) AS ?facebookVal)
           (SAMPLE(?instagram) AS ?instagramVal)
           (SAMPLE(?location) AS ?locationVal)
           (SAMPLE(?satSectionLabel) AS ?satSectionLabelVal)
           (SAMPLE(?satWebsite) AS ?satWebsiteVal)
           (SAMPLE(?websiteLang) AS ?websiteLangVal)
    WHERE {{
      ?id wdt:P6104 wd:Q134294510 ;
           wdt:P625 ?geo .

      OPTIONAL {{
        ?id wdt:P31 ?instance .
        ?instance wdt:P465 ?hexcolor .
        ?instance p:P1343 ?stmt .
        ?stmt pq:P1476 ?iconname .
      }}

      OPTIONAL {{ ?id rdfs:label ?labelLang FILTER (lang(?labelLang) = '{lang}') }}
      OPTIONAL {{ ?id schema:description ?descLang FILTER (lang(?descLang) = '{lang}') }}
      OPTIONAL {{ ?id rdfs:label ?svLabel FILTER (lang(?svLabel) = 'sv') }}
      OPTIONAL {{ ?id schema:description ?svdesc FILTER (lang(?svdesc) = 'sv') }}
      OPTIONAL {{ ?svwiki schema:about ?id; schema:isPartOf <https://sv.wikipedia.org/> }}
      OPTIONAL {{ ?id wdt:P18 ?img }}
      OPTIONAL {{ ?id wdt:P3749 ?p3749 }}
      OPTIONAL {{ ?id wdt:P856 ?website }}
      OPTIONAL {{ ?id wdt:P2013 ?facebook }}
      OPTIONAL {{ ?id wdt:P2003 ?instagram }}
      OPTIONAL {{ ?id wdt:P4173 ?location }}
      OPTIONAL {{
        ?id wdt:P2789 ?satSection .
        ?satSection rdfs:label ?satSectionLabel FILTER (lang(?satSectionLabel) = 'sv') .
        OPTIONAL {{
          ?satSection p:P856 ?websiteStmt .
          ?websiteStmt ps:P856 ?satWebsite .
          OPTIONAL {{ ?websiteStmt pq:P407 ?websiteLang }}
        }}
      }}
      {filter_clause}
    }}
    GROUP BY ?id ?geo ?hexcolor ?iconname
    """

    headers = {"Accept": "application/sparql-results+json"}
    r = requests.get(endpoint_url, params={'query': query}, headers=headers)
    r.raise_for_status()
    results = r.json()

    rows = []
    for item in results['results']['bindings']:
        label = item.get('labelLangVal', {}).get('value', '-')
        description = item.get('descLangVal', {}).get('value', '')
        sv_label = item.get('svLabelVal', {}).get('value', '')
        sv_description = item.get('svdescVal', {}).get('value', '')
        coords = item['geo']['value'].replace('Point(', '').replace(')', '')
        lon, lat = map(float, coords.split(' '))
        color = '#' + item.get('hexcolor', {}).get('value', '228b22')
        symbol = item.get('iconname', {}).get('value', 'landmark-JP')
        img = item.get('imgVal', {}).get('value', '')
        wiki_url = item.get('svwikiVal', {}).get('value', '')
        qid = item.get('id', {}).get('value', '').split('/')[-1]
        gmaps = item.get('p3749Val', {}).get('value', '')
        website = item.get('websiteVal', {}).get('value', '')
        facebook = item.get('facebookVal', {}).get('value', '')
        instagram = item.get('instagramVal', {}).get('value', '')
        location = item.get('locationVal', {}).get('value', '')
        sat_label = item.get('satSectionLabelVal', {}).get('value', '')
        sat_website = item.get('satWebsiteVal', {}).get('value', '')
        sat_lang = item.get('websiteLangVal', {}).get('value', '')

        popup_content = f"<b>{lang}: {label} - {description}</b><br>"
        if lang != 'sv' and (sv_label or sv_description):
            popup_content += f"<b>sv: {sv_label} - {sv_description}</b><br>"
        popup_content += "<div style='margin-top:8px;'><b>Links:</b><br>"
        popup_content += f"<a href='https://www.wikidata.org/wiki/{qid}' title='Wikidata link' target='_blank'>🔗 Wikidata</a><br>"
        if gmaps:
            popup_content += f"<a href='https://maps.google.com/?cid={gmaps}' title='View on Google Maps' target='_blank'>🗺️ Google Maps</a><br>"
        if wiki_url:
            popup_content += f"<a href='{wiki_url}' target='_blank' title='Wikipedia (sv)'>📘 Wikipedia (sv)</a><br>"
        if website:
            popup_content += f"🌐 <a href='{website}' target='_blank' title='Website'>{website}</a><br>"
        if facebook:
            popup_content += f"📘 <a href='https://facebook.com/{facebook}' target='_blank' title='Facebook'>{facebook}</a><br>"
        if instagram:
            popup_content += f"📸 <a href='https://instagram.com/{instagram}' target='_blank' title='Instagram'>{instagram}</a>"
            if location:
                popup_content += f" — <a href='https://www.instagram.com/explore/locations/{location}' target='_blank' title='Instagram Location'>{location}</a>"
            popup_content += "<br>"
        if img:
            popup_content += f"<br><div style='margin-top:8px;'><b>Image:</b><br><img src='{img}' width='200' title='{label}' alt='{label}'></div>"

        if sat_label:
            popup_content += f"<br><div style='margin-top:8px;'><b>SAT section:</b> {sat_label}"
            if sat_website:
                if 'en' in sat_lang:
                    popup_content += f" <a href='{sat_website}' target='_blank'>[en]</a>"
                elif 'sv' in sat_lang:
                    popup_content += f" <a href='{sat_website}' target='_blank'>[sv]</a>"
            popup_content += "</div>"

        rows.append({'label': popup_content, 'lat': lat, 'lon': lon, 'color': color, 'symbol': symbol})

    df = pd.DataFrame(rows)

    m = folium.Map(location=[59.3, 18.5], zoom_start=9)

    for _, row in df.iterrows():
        folium.CircleMarker(
            location=(row['lat'], row['lon']),
            radius=6,
            color=row['color'],
            fill=True,
            fill_color=row['color'],
            popup=row['label'],
            tooltip=row['label']
        ).add_to(m)

    display(m)

# Interactive update
widgets.interact(fetch_and_show_map, lang=language_selector, instance=instance_selector)


interactive(children=(Dropdown(description='Language:', index=37, options=(('Arabic - ar', 'ar'), ('Chinese - …

<function __main__.fetch_and_show_map(lang='en', instance='')>

### Add OSM trail SAT Arholma

19012436 --> https://www.openstreetmap.org/relation/19012436 

In [31]:
import requests
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output
from folium import GeoJson

# SPARQL endpoint
endpoint_url = "https://query.wikidata.org/sparql"

# Language groups
swedish_official = {'sv', 'fi', 'se', 'me', 'ri', 'sm', 'fit', 'yi', 'rmy'}
nordic_languages = swedish_official.union({'da', 'no', 'nn', 'is', 'fo'})
european_languages = {
    'en', 'fr', 'de', 'es', 'it', 'pt', 'pl', 'nl', 'cs', 'hu', 'ro', 'bg', 'el', 'tr', 'et', 'lv', 'lt', 'sk', 'sl', 'hr', 'mt',
    'da', 'fi', 'sv', 'no', 'is'
}  

# Language display mapping
language_names = {
    'sv': 'Swedish', 'en': 'English', 'fr': 'French', 'de': 'German', 'nn': 'Norwegian Nynorsk', 'nb': 'Norwegian Bokmål',
    'it': 'Italian', 'pl': 'Polish', 'es': 'Spanish', 'pt': 'Portuguese', 'ar': 'Arabic', 'ru': 'Russian', 'zh': 'Chinese',
    'fi': 'Finnish', 'da': 'Danish', 'nl': 'Dutch', 'jp': 'Japanese', 'fa': 'Persian', 'uk': 'Ukrainian', 'ku': 'Kurdish',
    'fit': 'Meänkieli', 'se': 'Northern Sami', 'sma': 'Southern Sami', 'smj': 'Lule Sami', 'sje': 'Pite Sami', 'sju': 'Ume Sami',
    'rmy': 'Romani', 'yi': 'Yiddish', 'et': 'Estonian', 'lv': 'Latvian', 'lt': 'Lithuanian', 'cs': 'Czech', 'hu': 'Hungarian',
    'el': 'Greek', 'tr': 'Turkish', 'ko': 'Korean', 'hi': 'Hindi', 'th': 'Thai', 'vi': 'Vietnamese', 'he': 'Hebrew',
    'id': 'Indonesian', 'ms': 'Malay', 'is': 'Icelandic', 'fo': 'Faroese'
}

all_languages = list(language_names.keys())

# Autodetect browser/system language with fallback to Swedish
import locale

def detect_language():
    try:
        lang = locale.getlocale()[0]
    except:
        lang = None
    if lang and lang[:2] in all_languages:
        return lang[:2]
    return 'sv'

# Dropdown for language selection with full names
language_selector = widgets.Dropdown(
    options=sorted([(f"{language_names[code]} - {code}", code) for code in all_languages]),
    value=detect_language(),
    description='Language:',
    disabled=False
)

# Instance selector (truncated for brevity, already populated above)
instance_selector = widgets.Dropdown(
    options=custom_instances,
    value='',
    description='Instance:',
    disabled=False,
)

def fetch_osm_trail_geojson():
    overpass_url = "https://overpass-api.de/api/interpreter"
    query = """
    [out:json];
    relation(19012436);
    >;
    out geom;
    """
    response = requests.get(overpass_url, params={'data': query})
    response.raise_for_status()
    data = response.json()

    geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    for element in data['elements']:
        if element['type'] == 'way' and 'geometry' in element:
            coords = [(pt['lat'], pt['lon']) for pt in element['geometry']]
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": [[lon, lat] for lat, lon in coords]
                },
                "properties": {
                    "name": element.get('tags', {}).get('name', 'SAT Trail Segment')
                }
            }
            geojson['features'].append(feature)
    return geojson

def fetch_and_show_map(lang='en', instance=''):
    # ... existing SPARQL and marker logic unchanged ...
    # [Insert SPARQL and popup code here as before.]

    m = folium.Map(location=[59.3, 18.5], zoom_start=9)

    # Add SAT Arholma trail line from OSM
    try:
        trail_geojson = fetch_osm_trail_geojson()
        GeoJson(trail_geojson, name="SAT Trail", style_function=lambda x: {
            'color': 'red', 'weight': 4, 'opacity': 0.8
        }).add_to(m)
    except Exception as e:
        print(f"Could not load trail geoline: {e}")

    for _, row in df.iterrows():
        folium.CircleMarker(
            location=(row['lat'], row['lon']),
            radius=6,
            color=row['color'],
            fill=True,
            fill_color=row['color'],
            popup=row['label'],
            tooltip=row['label']
        ).add_to(m)

    display(m)

# Interactive update
widgets.interact(fetch_and_show_map, lang=language_selector, instance=instance_selector)


interactive(children=(Dropdown(description='Language:', index=37, options=(('Arabic - ar', 'ar'), ('Chinese - …

<function __main__.fetch_and_show_map(lang='en', instance='')>

### Test with Marker

In [19]:
import folium

# Create the map centered on Finnhamn
finnhamn_map = folium.Map(location=[59.486, 18.807], zoom_start=13)

# Add POIs

# 🚰 Water pump
folium.Marker(
    location=[59.487, 18.809],
    popup="Drinking Water Pump",
    icon=folium.Icon(icon='tint', prefix='fa', color='cadetblue')
).add_to(finnhamn_map)

# 🍽️ Restaurant
folium.Marker(
    location=[59.4865, 18.806],
    popup="Svedtiljas Restaurant",
    icon=folium.Icon(icon='cutlery', prefix='fa', color='darkred')
).add_to(finnhamn_map)

# 🚴 Bike Rental
folium.Marker(
    location=[59.4863, 18.805],
    popup="Bike Rental at Svedtiljas",
    icon=folium.Icon(icon='bicycle', prefix='fa', color='blue')
).add_to(finnhamn_map)

# 🏨 Lodging
folium.Marker(
    location=[59.4858, 18.808],
    popup="Lotstornet Hotel",
    icon=folium.Icon(icon='bed', prefix='fa', color='green')
).add_to(finnhamn_map)

# 🧭 Trailhead
folium.Marker(
    location=[59.4872, 18.804],
    popup="Trail Start",
    icon=folium.Icon(icon='flag', prefix='fa', color='orange')
).add_to(finnhamn_map)

# 📍 Display the map inline
finnhamn_map


### Test with Legend

In [20]:
import folium
from branca.element import Template, MacroElement

# Create the map
finnhamn_map = folium.Map(location=[59.486, 18.807], zoom_start=13)

# Add your POIs (same as before)
folium.Marker(
    location=[59.487, 18.809],
    popup="Drinking Water Pump",
    icon=folium.Icon(icon='tint', prefix='fa', color='cadetblue')
).add_to(finnhamn_map)

folium.Marker(
    location=[59.4865, 18.806],
    popup="Svedtiljas Restaurant",
    icon=folium.Icon(icon='cutlery', prefix='fa', color='darkred')
).add_to(finnhamn_map)

folium.Marker(
    location=[59.4863, 18.805],
    popup="Bike Rental at Svedtiljas",
    icon=folium.Icon(icon='bicycle', prefix='fa', color='blue')
).add_to(finnhamn_map)

folium.Marker(
    location=[59.4858, 18.808],
    popup="Lotstornet Hotel",
    icon=folium.Icon(icon='bed', prefix='fa', color='green')
).add_to(finnhamn_map)

folium.Marker(
    location=[59.4872, 18.804],
    popup="Trail Start",
    icon=folium.Icon(icon='flag', prefix='fa', color='orange')
).add_to(finnhamn_map)

# 🧩 Add Custom Legend
legend_html = """
{% macro html(this, kwargs) %}

<div style="
    position: fixed;
    bottom: 25px;
    left: 25px;
    width: 160px;
    z-index: 9999;
    font-size:14px;
    background-color: white;
    border:2px solid gray;
    padding: 10px;
    box-shadow: 3px 3px 6px rgba(0,0,0,0.2);
">
<b>Map Legend</b><br>
<svg width="12" height="12"><circle cx="6" cy="6" r="5" fill="cadetblue"/></svg> Water Pump<br>
<svg width="12" height="12"><circle cx="6" cy="6" r="5" fill="darkred"/></svg> Restaurant<br>
<svg width="12" height="12"><circle cx="6" cy="6" r="5" fill="blue"/></svg> Bike Rental<br>
<svg width="12" height="12"><circle cx="6" cy="6" r="5" fill="green"/></svg> Lodging<br>
<svg width="12" height="12"><circle cx="6" cy="6" r="5" fill="orange"/></svg> Trail Start
</div>

{% endmacro %}
# 🧩 Add Custom Legend
legend_html = """
{% macro html(this, kwargs) %}

<div style="
    position: fixed;
    bottom: 25px;
    left: 25px;
    width: 160px;
    z-index: 9999;
    font-size:14px;
    background-color: white;
    border:2px solid gray;
    padding: 10px;
    box-shadow: 3px 3px 6px rgba(0,0,0,0.2);
">
<b>Map Legend</b><br>
<svg width="12" height="12"><circle cx="6" cy="6" r="5" fill="cadetblue"/></svg> Water Pump<br>
<svg width="12" height="12"><circle cx="6" cy="6" r="5" fill="darkred"/></svg> Restaurant<br>
<svg width="12" height="12"><circle cx="6" cy="6" r="5" fill="blue"/></svg> Bike Rental<br>
<svg width="12" height="12"><circle cx="6" cy="6" r="5" fill="green"/></svg> Lodging<br>
<svg width="12" height="12"><circle cx="6" cy="6" r="5" fill="orange"/></svg> Trail Start
</div>

{% endmacro %}
"""

legend = MacroElement()
legend._template = Template(legend_html)
finnhamn_map.get_root().add_child(legend)

# Display the map
finnhamn_map


### Add all sections ... 

* Shows all SAT trail segments via their OSM relations
* Adds tooltips with segment names
* Includes a layer control so you can toggle the SAT Trail layer on/off

Next steps could include:
* Sidebar listing trail names with zoom buttons
* Grouping trail segments by SAT section and coloring them differently
* Clickable links to the Wikidata item for each trail section



In [56]:
import requests
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output
from folium import GeoJson, GeoJsonTooltip

# SPARQL endpoint
endpoint_url = "https://query.wikidata.org/sparql"

# Language groups
swedish_official = {'sv', 'fi', 'se', 'me', 'ri', 'sm', 'fit', 'yi', 'rmy'}
nordic_languages = swedish_official.union({'da', 'no', 'nn', 'is', 'fo'})
european_languages = {
    'en', 'fr', 'de', 'es', 'it', 'pt', 'pl', 'nl', 'cs', 'hu', 'ro', 'bg', 'el', 'tr', 'et', 'lv', 'lt', 'sk', 'sl', 'hr', 'mt',
    'da', 'fi', 'sv', 'no', 'is'
}  

# Language display mapping
language_names = {
    'sv': 'Swedish', 'en': 'English', 'fr': 'French', 'de': 'German', 'nn': 'Norwegian Nynorsk', 'nb': 'Norwegian Bokmål',
    'it': 'Italian', 'pl': 'Polish', 'es': 'Spanish', 'pt': 'Portuguese', 'ar': 'Arabic', 'ru': 'Russian', 'zh': 'Chinese',
    'fi': 'Finnish', 'da': 'Danish', 'nl': 'Dutch', 'jp': 'Japanese', 'fa': 'Persian', 'uk': 'Ukrainian', 'ku': 'Kurdish',
    'fit': 'Meänkieli', 'se': 'Northern Sami', 'sma': 'Southern Sami', 'smj': 'Lule Sami', 'sje': 'Pite Sami', 'sju': 'Ume Sami',
    'rmy': 'Romani', 'yi': 'Yiddish', 'et': 'Estonian', 'lv': 'Latvian', 'lt': 'Lithuanian', 'cs': 'Czech', 'hu': 'Hungarian',
    'el': 'Greek', 'tr': 'Turkish', 'ko': 'Korean', 'hi': 'Hindi', 'th': 'Thai', 'vi': 'Vietnamese', 'he': 'Hebrew',
    'id': 'Indonesian', 'ms': 'Malay', 'is': 'Icelandic', 'fo': 'Faroese'
}

all_languages = list(language_names.keys())

# Autodetect browser/system language with fallback to Swedish
import locale

def detect_language():
    try:
        lang = locale.getlocale()[0]
    except:
        lang = None
    if lang and lang[:2] in all_languages:
        return lang[:2]
    return 'sv'

# Dropdown for language selection with full names
language_selector = widgets.Dropdown(
    options=sorted([(f"{language_names[code]} - {code}", code) for code in all_languages]),
    value=detect_language(),
    description='Language:',
    disabled=False
)

# Instance selector (truncated for brevity, already populated above)
instance_selector = widgets.Dropdown(
    options=custom_instances,
    value='',
    description='Instance:',
    disabled=False,
)

def fetch_osm_trail_geojson():
    overpass_url = "https://overpass-api.de/api/interpreter"
    query_wd = """
    SELECT ?relation WHERE {
      ?item wdt:P361 wd:Q131318799;
            wdt:P402 ?relation.
    }
"""

    headers = {"Accept": "application/sparql-results+json"}
    response = requests.get(endpoint_url, params={'query': query_wd}, headers=headers)
    response.raise_for_status()
    results = response.json()
    relation_ids = [int(binding['relation']['value']) for binding in results['results']['bindings']]
    query = (
    "[out:json];("
    + "".join([f"relation({rid});" for rid in relation_ids])
    + ");>;\nout geom;")
    response = requests.get(overpass_url, params={'data': query})
    response.raise_for_status()
    data = response.json()

    geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    for element in data['elements']:
        if element['type'] == 'way' and 'geometry' in element:
            coords = [(pt['lat'], pt['lon']) for pt in element['geometry']]
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": [[lon, lat] for lat, lon in coords]
                },
                "properties": {
                    "name": element.get('tags', {}).get('name', 'SAT Trail Segment')
                }
            }
            geojson['features'].append(feature)
    return geojson

def fetch_and_show_map(lang='en', instance=''):
    # ... existing SPARQL and marker logic unchanged ...
    # [Insert SPARQL and popup code here as before.]

    m = folium.Map(location=[59.3, 18.5], zoom_start=9)

    # Add SAT trail lines for all segments
    try:
        trail_geojson = fetch_osm_trail_geojson()
        GeoJson(
            trail_geojson,
            name="SAT Trail",
            style_function=lambda x: {
                'color': 'red', 'weight': 4, 'opacity': 0.8
            },
            tooltip=GeoJsonTooltip(fields=['name'], aliases=['Trail Segment'])
        ).add_to(m)
    except Exception as e:
        print(f"Could not load trail geoline: {e}")

    for _, row in df.iterrows():
        folium.CircleMarker(
            location=(row['lat'], row['lon']),
            radius=6,
            color=row['color'],
            fill=True,
            fill_color=row['color'],
            popup=row['label'],
            tooltip=row['label']
        ).add_to(m)

    folium.LayerControl().add_to(m)
    display(m)

# Interactive update
widgets.interact(fetch_and_show_map, lang=language_selector, instance=instance_selector)


interactive(children=(Dropdown(description='Language:', index=37, options=(('Arabic - ar', 'ar'), ('Chinese - …

<function __main__.fetch_and_show_map(lang='en', instance='')>

### Version 4

In [66]:
import requests
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output
from folium import GeoJson, GeoJsonTooltip

# SPARQL endpoint
endpoint_url = "https://query.wikidata.org/sparql"

# Language groups
swedish_official = {'sv', 'fi', 'se', 'me', 'ri', 'sm', 'fit', 'yi', 'rmy'}
nordic_languages = swedish_official.union({'da', 'no', 'nn', 'is', 'fo'})
european_languages = {
    'en', 'fr', 'de', 'es', 'it', 'pt', 'pl', 'nl', 'cs', 'hu', 'ro', 'bg', 'el', 'tr', 'et', 'lv', 'lt', 'sk', 'sl', 'hr', 'mt',
    'da', 'fi', 'sv', 'no', 'is'
}  

# Language display mapping
language_names = {
    'sv': 'Swedish', 'en': 'English', 'fr': 'French', 'de': 'German', 'nn': 'Norwegian Nynorsk', 'nb': 'Norwegian Bokmål',
    'it': 'Italian', 'pl': 'Polish', 'es': 'Spanish', 'pt': 'Portuguese', 'ar': 'Arabic', 'ru': 'Russian', 'zh': 'Chinese',
    'fi': 'Finnish', 'da': 'Danish', 'nl': 'Dutch', 'jp': 'Japanese', 'fa': 'Persian', 'uk': 'Ukrainian', 'ku': 'Kurdish',
    'fit': 'Meänkieli', 'se': 'Northern Sami', 'sma': 'Southern Sami', 'smj': 'Lule Sami', 'sje': 'Pite Sami', 'sju': 'Ume Sami',
    'rmy': 'Romani', 'yi': 'Yiddish', 'et': 'Estonian', 'lv': 'Latvian', 'lt': 'Lithuanian', 'cs': 'Czech', 'hu': 'Hungarian',
    'el': 'Greek', 'tr': 'Turkish', 'ko': 'Korean', 'hi': 'Hindi', 'th': 'Thai', 'vi': 'Vietnamese', 'he': 'Hebrew',
    'id': 'Indonesian', 'ms': 'Malay', 'is': 'Icelandic', 'fo': 'Faroese'
}

all_languages = list(language_names.keys())

# Autodetect browser/system language with fallback to Swedish
import locale

def detect_language():
    try:
        lang = locale.getlocale()[0]
    except:
        lang = None
    if lang and lang[:2] in all_languages:
        return lang[:2]
    return 'sv'

# Dropdown for language selection with full names
language_selector = widgets.Dropdown(
    options=sorted([(f"{language_names[code]} - {code}", code) for code in all_languages]),
    value=detect_language(),
    description='Language:',
    disabled=False
)

# Instance selector (truncated for brevity, already populated above)
instance_selector = widgets.Dropdown(
    options=custom_instances,
    value='',
    description='Instance:',
    disabled=False,
)

def fetch_osm_trail_geojson():
    overpass_url = "https://overpass-api.de/api/interpreter"
    query_wd = """
    SELECT ?relation WHERE {
      ?item wdt:P361 wd:Q131318799;
            wdt:P402 ?relation.
    }
"""

    headers = {"Accept": "application/sparql-results+json"}
    response = requests.get(endpoint_url, params={'query': query_wd}, headers=headers)
    response.raise_for_status()
    results = response.json()
    relation_ids = [int(binding['relation']['value']) for binding in results['results']['bindings']]

    query = (
    "[out:json];("
    + "".join([f"relation({rid});" for rid in relation_ids])
    + ");>;\nout geom;")    
    response = requests.get(overpass_url, params={'data': query})
    response.raise_for_status()
    data = response.json()

    geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    for element in data['elements']:
        if element['type'] == 'way' and 'geometry' in element:
            coords = [(pt['lat'], pt['lon']) for pt in element['geometry']]
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": [[lon, lat] for lat, lon in coords]
                },
                "properties": {
                    "name": element.get('tags', {}).get('name', 'SAT Trail Segment')
                }
            }
            geojson['features'].append(feature)
    return geojson

from branca.element import MacroElement
from jinja2 import Template

def fetch_and_show_map(lang='en', instance=''):
    # ... existing SPARQL and marker logic unchanged ...
    # [Insert SPARQL and popup code here as before.]

    m = folium.Map(location=[59.3, 18.5], zoom_start=9)

    # Add SAT trail lines for all segments
    try:
        trail_geojson = fetch_osm_trail_geojson()
        GeoJson(
            trail_geojson,
            name="SAT Trail",
            style_function=lambda x: {
                'color': 'red', 'weight': 4, 'opacity': 0.8
            },
            tooltip=GeoJsonTooltip(fields=['name'], aliases=['Trail Segment'])
        ).add_to(m)
    except Exception as e:
        print(f"Could not load trail geoline: {e}")

    for _, row in df.iterrows():
        folium.CircleMarker(
            location=(row['lat'], row['lon']),
            radius=6,
            color=row['color'],
            fill=True,
            fill_color=row['color'],
            popup=row['label'],
            tooltip=row['label']
        ).add_to(m)

    folium.LayerControl().add_to(m)

    # 🧩 Add Custom Legend
    legend_html = """
    {% macro html(this, kwargs) %}
    <div style="
        position: fixed;
        bottom: 25px;
        left: 25px;
        width: 180px;
        z-index: 9999;
        font-size:14px;
        background-color: white;
        border:2px solid gray;
        padding: 10px;
        box-shadow: 3px 3px 6px rgba(0,0,0,0.2);
    ">
    <b>Map Legend</b><br>
    <svg width='12' height='12'><circle cx='6' cy='6' r='5' fill='blue'/></svg> Bike Rental<br>
    <svg width='12' height='12'><circle cx='6' cy='6' r='5' fill='cadetblue'/></svg> Drinking Water<br>
    <svg width='12' height='12'><circle cx='6' cy='6' r='5' fill='orange'/></svg> Grocery<br>
    <svg width='12' height='12'><circle cx='6' cy='6' r='5' fill='darkorange'/></svg> Kayak Rental<br>
    <svg width='12' height='12'><circle cx='6' cy='6' r='5' fill='green'/></svg> Lodging<br>
    <svg width='12' height='12'><circle cx='6' cy='6' r='5' fill='darkred'/></svg> Restaurant<br>
    <svg width='12' height='12'><circle cx='6' cy='6' r='5' fill='purple'/></svg> Museum<br>
    <svg width='12' height='12'><circle cx='6' cy='6' r='5' fill='gray'/></svg> Misc<br>
    </div>
    {% endmacro %}
    """
    legend = MacroElement()
    legend._template = Template(legend_html)
    m.get_root().add_child(legend)
    display(m)

# Interactive update
widgets.interact(fetch_and_show_map, lang=language_selector, instance=instance_selector)


interactive(children=(Dropdown(description='Language:', index=37, options=(('Arabic - ar', 'ar'), ('Chinese - …

<function __main__.fetch_and_show_map(lang='en', instance='')>

In [64]:
import requests
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output
from folium import GeoJson, GeoJsonTooltip

# SPARQL endpoint
endpoint_url = "https://query.wikidata.org/sparql"

# Language groups
swedish_official = {'sv', 'fi', 'se', 'me', 'ri', 'sm', 'fit', 'yi', 'rmy'}
nordic_languages = swedish_official.union({'da', 'no', 'nn', 'is', 'fo'})
european_languages = {
    'en', 'fr', 'de', 'es', 'it', 'pt', 'pl', 'nl', 'cs', 'hu', 'ro', 'bg', 'el', 'tr', 'et', 'lv', 'lt', 'sk', 'sl', 'hr', 'mt',
    'da', 'fi', 'sv', 'no', 'is'
}  

# Language display mapping
language_names = {
    'sv': 'Swedish', 'en': 'English', 'fr': 'French', 'de': 'German', 'nn': 'Norwegian Nynorsk', 'nb': 'Norwegian Bokmål',
    'it': 'Italian', 'pl': 'Polish', 'es': 'Spanish', 'pt': 'Portuguese', 'ar': 'Arabic', 'ru': 'Russian', 'zh': 'Chinese',
    'fi': 'Finnish', 'da': 'Danish', 'nl': 'Dutch', 'jp': 'Japanese', 'fa': 'Persian', 'uk': 'Ukrainian', 'ku': 'Kurdish',
    'fit': 'Meänkieli', 'se': 'Northern Sami', 'sma': 'Southern Sami', 'smj': 'Lule Sami', 'sje': 'Pite Sami', 'sju': 'Ume Sami',
    'rmy': 'Romani', 'yi': 'Yiddish', 'et': 'Estonian', 'lv': 'Latvian', 'lt': 'Lithuanian', 'cs': 'Czech', 'hu': 'Hungarian',
    'el': 'Greek', 'tr': 'Turkish', 'ko': 'Korean', 'hi': 'Hindi', 'th': 'Thai', 'vi': 'Vietnamese', 'he': 'Hebrew',
    'id': 'Indonesian', 'ms': 'Malay', 'is': 'Icelandic', 'fo': 'Faroese'
}

all_languages = list(language_names.keys())

# Autodetect browser/system language with fallback to Swedish
import locale

def detect_language():
    try:
        lang = locale.getlocale()[0]
    except:
        lang = None
    if lang and lang[:2] in all_languages:
        return lang[:2]
    return 'sv'

# Dropdown for language selection with full names
language_selector = widgets.Dropdown(
    options=sorted([(f"{language_names[code]} - {code}", code) for code in all_languages]),
    value=detect_language(),
    description='Language:',
    disabled=False
)

# Instance selector (truncated for brevity, already populated above)
instance_selector = widgets.Dropdown(
    options=custom_instances,
    value='',
    description='Instance:',
    disabled=False,
)

def fetch_osm_trail_geojson():
    overpass_url = "https://overpass-api.de/api/interpreter"
    query_wd = """
    SELECT ?relation WHERE {
      ?item wdt:P361 wd:Q131318799;
            wdt:P402 ?relation.
    }
"""

    headers = {"Accept": "application/sparql-results+json"}
    response = requests.get(endpoint_url, params={'query': query_wd}, headers=headers)
    response.raise_for_status()
    results = response.json()
    relation_ids = [int(binding['relation']['value']) for binding in results['results']['bindings']]
    query = (
    "[out:json];("
    + "".join([f"relation({rid});" for rid in relation_ids])
    + ");>;\nout geom;")    

    response = requests.get(overpass_url, params={'data': query})
    response.raise_for_status()
    data = response.json()

    geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    for element in data['elements']:
        if element['type'] == 'way' and 'geometry' in element:
            coords = [(pt['lat'], pt['lon']) for pt in element['geometry']]
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": [[lon, lat] for lat, lon in coords]
                },
                "properties": {
                    "name": element.get('tags', {}).get('name', 'SAT Trail Segment')
                }
            }
            geojson['features'].append(feature)
    return geojson

from branca.element import MacroElement
from jinja2 import Template

def fetch_and_show_map(lang='en', instance=''):
    # ... existing SPARQL and marker logic unchanged ...
    # [Insert SPARQL and popup code here as before.]

    m = folium.Map(location=[59.3, 18.5], zoom_start=9)

    # Add SAT trail lines for all segments
    try:
        trail_geojson = fetch_osm_trail_geojson()
        GeoJson(
        trail_geojson,
        name="SAT Trail",
        style_function=lambda x: {
            'color': 'red', 'weight': 4, 'opacity': 0.8
        },
        tooltip=GeoJsonTooltip(
            fields=['name'],
            aliases=['SAT Section'],
            sticky=True,
            direction='top',
            opacity=0.9
        )
    ).add_to(m)
    except Exception as e:
        print(f"Could not load trail geoline: {e}")

    for _, row in df.iterrows():
        folium.CircleMarker(
            location=(row['lat'], row['lon']),
            radius=6,
            color=row['color'],
            fill=True,
            fill_color=row['color'],
            popup=row['label'],
            tooltip=row['label']
        ).add_to(m)

    folium.LayerControl().add_to(m)

    # 🧩 Add Custom Legend
    legend_html = """
    {% macro html(this, kwargs) %}
    <div style="
        position: fixed;
        bottom: 25px;
        left: 25px;
        width: 160px;
        z-index: 9999;
        font-size:14px;
        background-color: white;
        border:2px solid gray;
        padding: 10px;
        box-shadow: 3px 3px 6px rgba(0,0,0,0.2);
    ">
    <b>Map Legend</b><br>
    <svg width='12' height='12'><circle cx='6' cy='6' r='5' fill='cadetblue'/></svg> Water Pump<br>
    <svg width='12' height='12'><circle cx='6' cy='6' r='5' fill='darkred'/></svg> Restaurant<br>
    <svg width='12' height='12'><circle cx='6' cy='6' r='5' fill='blue'/></svg> Bike Rental<br>
    <svg width='12' height='12'><circle cx='6' cy='6' r='5' fill='green'/></svg> Lodging<br>
    <svg width='12' height='12'><circle cx='6' cy='6' r='5' fill='orange'/></svg> Trail Start
    </div>
    {% endmacro %}
    """
    legend = MacroElement()
    legend._template = Template(legend_html)
    m.get_root().add_child(legend)
    display(m)

# Interactive update
widgets.interact(fetch_and_show_map, lang=language_selector, instance=instance_selector)


interactive(children=(Dropdown(description='Language:', index=37, options=(('Arabic - ar', 'ar'), ('Chinese - …

<function __main__.fetch_and_show_map(lang='en', instance='')>